In [1]:
# requirements

# !pip install requests
# !pip install time
# !pip install pandas
# !pip install numpy
# !pip install matplotlib.pyplot
# !pip install schedule
!pip install plotly




# Introduction
Cryptocurrency markets are fast-moving, data-heavy, and often difficult for beginners to make sense of. Prices move by the second, market caps constantly fluctuate, and trading volume can spike unpredictably. For a new investor and beginner Python user, answering a simple question like “Which coin should I buy?” requires more than just looking at a chart. It requires understanding how to gather real data, clean and structure it properly, visualize trends, and compute risk-based metrics.

This tutorial guides users through that entire process step-by-step. We will begin by pulling real historical cryptocurrency data directly from the CoinGecko API. Then we will transform the raw JSON responses into clean and structured time-series data that Python can analyze. After preparing daily key metrics tables for Bitcoin, Ethereum, and Dogecoin, we will visualize price trends using Plotly and compute financial indicators such as risk/return ratios and volatility. Finally, we will bring together everything the user learned in an interactive “buy or sell” simulation that encourages real decision-making.

Our goal is not just to teach users how to write code, it is to help them understand the workflow behind market analysis.
By the end of the tutorial, users will have hands-on experience using Python to explore real financial data and evaluate assets based on evidence.

# Section 1: Calling CoinGecko's API

An Application Programming Interface (API) is basically a set of rules of how other people can interact with a service. In this tutorial, we are going to request historial data on different cryptocurrineces using Python's request library.


## Step 1 - API URL

CoinGecko documentation: https://docs.coingecko.com/reference/coins-id-market-chart-range

The request library in Python sends a request to an URL (API Endpoint) for its data. Based on the documentation, an endpoint that allows us to get historical data for a certain coin is at this URL: "https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"


Looking at the URL, it contains the parameter: {coin_id}. Coin_id is the name of the cryptocurrency that we want the historical data for. For this example lets use bitcoin. CoinGecko has historical market data for almost every coin. The {coin_id} is usually the name of the coin in all lowercase. However this google sheet(https://docs.google.com/spreadsheets/d/1wTTuxXt8n9q7C4NDXqQpI3wpKu1_5bGVmP9Xz0XGSyU/edit?gid=0#gid=0) has the link of all of the {coin_id}, if you have trouble finding a coin_id for a certain coin.



In [2]:
import requests

coin_id = "bitcoin"

url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

response = requests.get(url)

data = response.json()

print(data)

{'status': {'timestamp': '2025-12-11T02:06:08.512Z', 'error_code': 10014, 'error_message': "Invalid value for 'to' and 'from' provided for the date parameter. Use YYYY-MM-DD, YYYY-MM-DDTHH:MM or UNIX timestamp only."}}


## Step 2 - Query Parameters

After running the code cell above, you might have noticed that the request errored because there was an invalid value for "to" and "from". This is because on top of giving the API Endpoint a coin_id, we also need to input what currency we want the data in and from what time range. These required parameters can be found in the documentation.

<img src="https://github.com/jheimann05/JPMCrypto/blob/main/api_param_img.png?raw=1" alt="title" width="50%">

For the currency, we can use the example that they give us and input it as USD. The documentation states that the date ranges can be a UNIX timestamp. We want the most recent 3 months of data. To get the most recent 3 months as a UNIX timestamp we can use the time python library. time.time() returns the current time as a UNIX timestamp. Then we can calculate the number of seconds in 3 months (90 * 24 * 60 * 60) and subtract that from the current time to get the UNIX timestamp of 3 months ago from right now



In [3]:
import requests
import time

coin_id = "bitcoin"

url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

to_ts = int(time.time())
from_ts = to_ts - 90 * 24 * 60 * 60

params = {
    "vs_currency": "usd",
    "from": from_ts,
    "to": to_ts
}

response = requests.get(url, params=params)
data = response.json()
first_ten = {data_type: data[data_type][:10] for data_type in data}

print(first_ten)

{'prices': [[1757646140659, 115321.70299565145], [1757649743377, 115198.04689576305], [1757653320681, 115430.88537282546], [1757656967846, 115656.41297780728], [1757660585554, 115339.65347651657], [1757664157781, 115143.92060273138], [1757667764701, 115090.51727175506], [1757671350254, 114977.79702489631], [1757674945697, 114970.87662702953], [1757678476911, 114944.79334030187]], 'market_caps': [[1757646140659, 2299596299684.7803], [1757649743377, 2295138870194.161], [1757653320681, 2298501887155.019], [1757656967846, 2303582766450.0186], [1757660585554, 2296893948549.9297], [1757664157781, 2293156405143.4966], [1757667764701, 2292369656438.616], [1757671350254, 2290229958319.334], [1757674945697, 2290338188719.942], [1757678476911, 2289576604539.2666]], 'total_volumes': [[1757646140659, 49310735354.19669], [1757649743377, 49484083557.887856], [1757653320681, 49223007403.148895], [1757656967846, 49716272610.759], [1757660585554, 49380350028.38388], [1757664157781, 49958661649.62895], [

## Step 3 - Multiple Coins

In order to see the historical market data for different coins, all we have to change is the coin id based on the list linked above. For this tutorial we will also look at etherum and dogecoin in addition to bitcoin. In order to avoid repeated code, we wrapped the code in a function with an argument for the coin id.

In [4]:
import requests
import time

def fetch_data(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

    to_ts = int(time.time())
    from_ts = to_ts - 90 * 24 * 60 * 60

    params = {
        "vs_currency": "usd",
        "from": from_ts,
        "to": to_ts
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data

btc_data = fetch_data("bitcoin")
eth_data = fetch_data("ethereum")
doge_data = fetch_data("dogecoin")

btc_first_ten = {data_type: btc_data[data_type][:10] for data_type in btc_data}
eth_first_ten = {data_type: eth_data[data_type][:10] for data_type in eth_data}
doge_first_ten = {data_type: doge_data[data_type][:10] for data_type in doge_data}

print(btc_first_ten)
print(eth_first_ten)
print(doge_first_ten)


{'prices': [[1757646140659, 115321.70299565145], [1757649743377, 115198.04689576305], [1757653320681, 115430.88537282546], [1757656967846, 115656.41297780728], [1757660585554, 115339.65347651657], [1757664157781, 115143.92060273138], [1757667764701, 115090.51727175506], [1757671350254, 114977.79702489631], [1757674945697, 114970.87662702953], [1757678476911, 114944.79334030187]], 'market_caps': [[1757646140659, 2299596299684.7803], [1757649743377, 2295138870194.161], [1757653320681, 2298501887155.019], [1757656967846, 2303582766450.0186], [1757660585554, 2296893948549.9297], [1757664157781, 2293156405143.4966], [1757667764701, 2292369656438.616], [1757671350254, 2290229958319.334], [1757674945697, 2290338188719.942], [1757678476911, 2289576604539.2666]], 'total_volumes': [[1757646140659, 49310735354.19669], [1757649743377, 49484083557.887856], [1757653320681, 49223007403.148895], [1757656967846, 49716272610.759], [1757660585554, 49380350028.38388], [1757664157781, 49958661649.62895], [

# Section 2: Cleaning and Structuring Data
In the previous section, we wrote the `fetch_data(coin_id)` function that calls the CoinGecko API and returns raw historical data for a cryptocurrency.

The good news: this raw data already has a lot of useful information.
The bad news: it's not in a form we can easily analyze.

In this section, we'll turn that raw JSON into a clean, daily table with:
- Open, High, Low, Close price (OHLC)
- Daily trading volume
- Daily market cap

We'll do this for multiple coins, so later we can compare them.





## Step 1 - Inspecting What the API Actually Returns
Before writing any cleaning code, it's essential to examine the shape of the raw data. This is good practice anytime you work with an API. We will do this by looking at the "keys" of the API which is basically the metrics that we can pull.

In [5]:
sample = fetch_data("bitcoin")
print(sample.keys())

dict_keys(['status'])


You should see something like: `dict_keys(['prices', 'market_caps', 'total_volumes'])`

So the API is giving us three separate time series:
- prices — historical prices
- market_caps — historical market capitalization
- total_volumes — historical trading volume

A time series is a sequence of data points collected over time, where each value is associated with a specific timestamp. In finance, almost everything — prices, volume, market cap — is recorded as a time series, because we care about how these numbers change over time.

Each of the time series we get from CoinGecko's API is provided as a list of pairs:

In [6]:
for key in sample.keys():
    print(f"\n--- {key} ---")
    print(sample[key][:3])  # look at first 3 entries


--- status ---


KeyError: slice(None, 3, None)

All three have the same basic format:
- The first element: a timestamp in milliseconds
- The second element: the value at that time

This is beneficial because it allows us to process all three with the same helper function.
## Step 2 - Converting Raw Lists into a Clean Time Series

Right now, each series is just a Python list of lists.
Our goal is to turn it into a pandas DataFrame that has a proper human-readable datetime column, has a clearly named numeric value column, is sorted in true chronological order, handles missing, zero, or malformed values, and is easy to merge with the other time series later

A DataFrame is perfect for this because it's the standard structure for time-series analysis in Python. It lets us sort, merge, resample, and visualize data much more easily than raw lists.

We can write one helper function that does all of this:

In [7]:
import pandas as pd
import numpy as np

def _to_df(series, value_name):
    # Turn raw list into a table with two columns: ts_ms and the value
    df = pd.DataFrame(series, columns=["ts_ms", value_name])

    # Convert timestamp from milliseconds → actual datetime in UTC
    df["date_utc"] = pd.to_datetime(df["ts_ms"], unit="ms", utc=True)

    # Drop the raw timestamp column now that we have a readable datetime
    df = df.drop(columns=["ts_ms"]).sort_values("date_utc")

    # Ensure the value column is numeric, and treat 0 as missing (often signals bad data)
    # Crypto APIs often report 0 when data is unavailable, not when the real value is zero.
    df[value_name] = (
        pd.to_numeric(df[value_name], errors="coerce")
          .replace(0, np.nan)
    )

    return df


After running this helper, each of the three datasets (`prices, market_caps, total_volumes`) becomes a clean, sorted time series with real timestamps instead of unreadable integers. This prepares them for the next step: merging price, volume, and market cap so they can be compared on the same timeline.

In [8]:
prices_df    = _to_df(sample["prices"], "price")
mcap_df      = _to_df(sample["market_caps"], "mcap")
volumes_df   = _to_df(sample["total_volumes"], "vol_ccy")

prices_df.head()

KeyError: 'prices'

## Step 3 - Aligning Prices, Market Cap, and Volume in Time
Each time series we have (price, market cap, volume) is recorded independently, so their timestamps might not line up perfectly. For example we might have:
- A price at 10:00:00.123
- A market cap at 10:00:05.456
- A volume at 10:01:01.789

But for analysis, we want one combined table where each row has price, market cap, and volume at approximately the same time.

To do that, we'll align the three series onto a shared time axis.

To do this, we use pandas.merge_asof(), which is like a time-aware join: it matches each row with the closest timestamp from the other table.

### Using ```merge_asof``` to match time
Pandas gives us a helpful function called `merge_asof`. You can think of it as a joining two tables by timestamp, but if the timestamps don't match exaclty, then use the nearest one in time.

We will use it to join:


1.   Prices and market caps
2.   Then that result and volumes

Here's the helper function that we will use to do this:

In [10]:
def align_all_series(data_json):
    # Convert each raw list into its own cleaned DataFrame
    prices_df  = _to_df(data_json["prices"], "price")
    mcap_df    = _to_df(data_json["market_caps"], "mcap")
    volumes_df = _to_df(data_json["total_volumes"], "vol_ccy")

    # Merge prices with market caps based on nearest timestamp
    merged = pd.merge_asof(
        prices_df.sort_values("date_utc"),
        mcap_df.sort_values("date_utc"),
        on="date_utc",
        direction="nearest",
        tolerance=pd.Timedelta("5min"),
    )

    # Merge in volumes the same way
    merged = pd.merge_asof(
        merged.sort_values("date_utc"),
        volumes_df.sort_values("date_utc"),
        on="date_utc",
        direction="nearest",
        tolerance=pd.Timedelta("5min"),
    )

    # Clean up small gaps and drop unusable rows
    merged = merged.ffill(limit=3).dropna(subset=["price", "mcap", "vol_ccy"])

    # Use datetime as index for easy resampling later
    merged = merged.set_index("date_utc").sort_index()

    return merged


What those options mean:
- `direction="nearest"`: For each timestamp in the left table, find the closest timestamp in the right table.
- `tolerance=pd.Timedelta("5min")`: Only match values if they are within 5 minutes of each other.
If the nearest point is farther away than that, it won't be matched.
- `ffill(limit=3)`: “Forward fill” small gaps: if we're missing a value for a short stretch,
reuse the last known value for up to 3 rows; this is common for cleaning tiny gaps.
- `.dropna(subset=["price", "mcap", "vol_ccy"])`: After merging and filling, we still drop any rows where any critical field is still missing, because those rows could distort our analysis.

Now we have a high-frequency time series where each row has price, market cap, and volume aligned on the same timeline.

We can test and see this:

In [11]:
aligned_tick = align_all_series(sample)
aligned_tick.head()

KeyError: 'prices'

You should now see one table with columns like `price`, `mcap`, and `vol_ccy`, all indexed by a single clean datetime column — which sets us up for the next step:

## Step 4 - Resampling to Daily OHLCV and Market Cap
Right now, we have a table, `aligned_tick`, where each row represents a moment in time — often every few minutes. This is good for very detailed analysis, but it is too noisy to compare assets or compute risk metrics like we want to do.

Financial analysts typically work with daily data, because


*   It smooths random noise from minute-to-minute price movements
*   Assets become easier to compare
*   Indicators like volatility and returns become more stable
*   Plotting trends becomes clearer


To convert our minute-level time series into daily data, we use a financial format we use a classic financial format called OHLCV:


Some typical finance key terms and indicators in the market that we will use are
- Open — The first recorded price of the day
- High — The highest price that occurred during that day
- Low — The lowest price observed during that day
- Close — The final price of the day (most widely used in analysis)
- Volume — Total trading activity during that day (sum of all intraday volume measurements)

We will also include the market cap of the day which is the total value of the entire asset(Price * number of coins in circulation) at the end of that day.

This data is the backbone of most trading platforms, charting systems, and portfolio analytics.


---



###Turning High-Frequency Data into Daily Summary Rows
Pandas makes this very easy with `.resample("1D")`, which groups the dataset by day instead of by individual timestamps.

Here is the full function:


In [12]:
def coingecko_to_daily_table(data_json):
    # Get high-frequency aligned data
    tick = align_all_series(data_json)

    # Daily OHLC for price
    ohlc = tick["price"].resample("1D").agg(
        Open="first",
        High="max",
        Low="min",
        Close="last"
    )

    # Daily market cap: we take the last value in each day (end-of-day cap)
    daily_mcap = tick["mcap"].resample("1D").agg(
        Market_Cap="last"
    )

    # Daily volume in currency: sum of intraday volumes
    daily_vol = tick["vol_ccy"].resample("1D").agg(
        Volume_Currency="sum"
    )

    # Combine everything into one table
    daily = pd.concat([ohlc, daily_mcap, daily_vol], axis=1)
    daily.index.name = "Date"

    # Drop empty days
    daily = daily.dropna(how="all")

    return daily


You should now see a nice table that has all key metrics grouped togeather by date. This is now a true financial time series, ready for plotting, comparing currencies, computing returns, measuring volatility, and performing risk analysis.

## Step 5 - Running the Pipeline for Multiple Coins
Now that we've built resuable functions(`fetch_data()`, `_to_df()`, `align_all_series()`, and `coingecko_to_daily_table()`) we don't need to rewrite any code to process another cryptocurrency.
All of our cleaning steps work the same way for any asset that CoinGecko provides. This is one of the biggest benefits of writing reusable helper functions: Once the process works for one coin, it works for all of them.

Now we can pick the coins we want to analyze, loop through each one, and build its daily table.

In [13]:
coins = ["bitcoin", "ethereum", "dogecoin"]

daily_data = {}
for coin in coins:
    # Pull raw API data
    raw = fetch_data(coin)
    # Clean, align, and resample
    daily = coingecko_to_daily_table(raw)
    # Store in our dictionary
    daily_data[coin] = daily

daily_data["ethereum"].head()

KeyError: 'prices'

Now `daily_data` is a dictionary of full daily data tables, one for each asset:
- `daily_data["bitcoin"]` is BTC's daily OHLCV + market cap
- `daily_data["ethereum"]` is ETH's daily table
- `daily_data["dogecoin"]` is DOGE's table

All have the same strucutre, which is extremely useful because it means we can
1. Compare returns
2. Measure volatility differences
3. Plot the assets on the same chart
4. Calculate risk-reward ratios
5. Run the same analysis functions for every asset

This section has set the stage for the rest of the tutorial and we are now equipped to start analyzing.

# Section 3: Graphing Daily Price

In the previous section, we cleaned the data from the API request. The data is now in the form of a dictionary where the  daily_data is a dictionary. The dictionary's index are the coin names and its values is a pandas dataframe of the coins daily OHLC, market cap, and volume.

In the this section we will use plotly to graph the daily closing price for the coins.

## Step 1 - Isolating Daily Closing Price and their Dates

In order to create a scatter plot, where the x-axis is the date and the y-axis are the closing prices for the coins, we need to isolate the daily closing prices in the dataframe. First using the btc_data dictionary from the last section, we grab the bitcoin dataframe from it. Then, we grab just the "Close" column from the dataframe and convert it into a list. Finally, we convert the dataframe indexes (which are dates) into a list.

This leaves us with two lists of the daily close prices and their dates.

In [14]:
btc_data = daily_data["bitcoin"]

close_prices = btc_data["Close"].to_list()

dates = btc_data.index.to_list()

close_prices[:10]

KeyError: 'bitcoin'

## Step 2 - Creating a Plotly Scatter Plot

Now that we have a list for the axis, we can create our plotly scatter plot. Breaking down the plot_close_prices function, fig = go.Figure() creates an emptly plotly figure. A plotly figure is our graph object. Then we add a trace which adds a datasets to the plotly figure, which in our case is the daily closing prices and their dates. The "lines+markers" mode tells plotly to create a continous line connecting every point(line) and a marker dot for each date(marker)

In [15]:
import plotly.graph_objects as go

def plot_close_prices(close_prices, dates, coin="bitcoin"):

    fig = go.Figure()

    coin = coin.capitalize()

    fig.add_trace(

        # creates a scatter plot
        go.Scatter(
            x=dates,
            y=close_prices,
            mode="lines+markers",      # line + scatter points
            name=f"{coin} Close Price"
        )
    )

    fig.update_layout(
        # set the axis title and chart title
        title=f"{coin} Daily Close Price",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        hovermode="x unified"
    )

    fig.show()

plot_close_prices(close_prices, dates)

NameError: name 'close_prices' is not defined

## Step 3 - Doing this for multiple Coins

We follow the same process of isolating the close prices/dates and creating a trace using the two lists as our x and y axis datasets for each coin. In plotly, its simple to have multiple lines on the same graph. All you have to do is add multiple traces to the same plotly figure object.

In [16]:
import plotly.graph_objects as go

coins = ["bitcoin", "ethereum", "dogecoin"]

fig = go.Figure()

for coin in coins:
    df = daily_data[coin]
    dates = df.index.to_list()
    closes = df["Close"].to_list()

    fig.add_trace(
        go.Scatter(
            x=dates,
            y=closes,
            mode="lines+markers",
            name=f"{coin.capitalize()}"
        )
    )

fig.update_layout(
    title="Daily Close Prices for Multiple Cryptocurrencies",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    hovermode="x unified"
)

fig.show()

KeyError: 'bitcoin'

# Section 4: Risk/Return and Volatitly

## Step 1 - Analyzing Risk/Return of a Coin


Looking at the risk/return ratio of a coin helps us make a more informed buying decision.


Our risk/return function uses the Sharpe Ratio, which is a widely-accepted formula for evaluating the risk of buying a cryptocurrency. The function takes in an array of prices and a risk-free return value. The risk free-rate (rf) is the percentage return you can earn by investing in a Treasury Bill instead of the cryptocurrency being analyzed. As of November 16, 2025, the rf is 3.89%. You can find this value by going to the US department of treasury website.


In our function body, we first calculate the daily returns and store these values in an array called returns. With that, we can calculate the risk by taking the standard deviation of the returns array and calculate the reward by taking the mean of the returns array. The final risk/reward ratio is calculated by subtracting the risk-free rate from the reward, then dividing that value by the risk.


In [76]:
import numpy as np
import math

# Sharp Ratio Grading Thresholds:
# Less than 0: high risk/low reward
# 0.0 - 0.99: low risk/low reward
# 1.0 – 1.99: Adequate/good
# Greater than 2: Very high reward potential

# prices: array of prices
# rf: risk-free rate of return
def risk_return(prices, rf=0.04):
    returns = [((prices[i] - prices[i-1]) / prices[i-1]) for i in range(1, len(prices))]
    risk = np.std(returns)
    reward = np.mean(returns)
    return np.round((reward - rf / risk), 2)

Lets look at the risk/reward ratio for bitcoin. Lets grab bitcoin's 90 day closing price data from the daily_data data frame and call our risk_return function on the column 'Close', which contains the closing prices.

In [77]:
btc_closing_prices = daily_data['bitcoin']['Close']
print(risk_return(btc_closing_prices))

-1.83


/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_64237/3999890648.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



A negative risk/reward value indicates this investment should be avoided because it is very risky and has low reward potential. Risk/reward ratios in the 0.0-0.99 range indicate a safe investment. Most investments fall in the 1.0-1.99 range and these offer a balance between risk and reward. A ratio above 2 suggests use of leverage to inflate returns, which also increases the risk. Thus, it is encouraged that investors conduct further research on such investments.


## Step 2 - Volatility







The volatility function shows how frequently a cryptocurrency price moves. Our function takes in an array of prices as input and an integer n that represents the number of days we want to check volatility for. To calculate daily volatility, we will use a 30-day window of past prices, which means the value of n cannot exceed 60, since we have data for 90 days.

In our volatility function, we will again first calculate the returns (like we did for the risk/return function). We also calculate the average of the returns. We then create an empty array called crypto_volatility, where we will store our daily volatility values. We will create a nested for loop. Each outer loop iteration represents the volatility for one day. The inner loop calculates the volatility for that day using the daily price values from the past 30 days. The volatility for that day is then calculated by using the standard deviation formula. We convert the volatility to a percentage value by multiplying by 100 and make it more readable by using python's round() function. We then store the daily volatility value in our crypto_volatility array.


In [78]:
# n represents number of days. The value of n cannot be above 60 since we are using a 30-day window to calculate the volatility and we have data for 90 days.
def volatility(prices, n, window=30):
    returns = [((prices[i] - prices[i-1]) / prices[i-1]) for i in range(1, len(prices))]
    crypto_volatility = []
    for i in range(90-n, 90):
        window_returns = returns[i-window:i]
        average = np.mean(window_returns)
        ssd = 0
        for r in window_returns:
            ssd += (r - average)**2 # sum of squared deviations
        daily_volatility = round(math.sqrt(ssd / window) * 100, 2)
        crypto_volatility.append(daily_volatility)
    return crypto_volatility



Lets now look at volatility for bitcoin in the past 60 days. This time, lets use the opening prices.

In [79]:
btc_opening_prices = daily_data['bitcoin']['Open']
btc_volatility = volatility(btc_opening_prices, 60, 30)
print(btc_volatility)

[2.03, 2.16, 2.16, 2.18, 2.22, 2.23, 2.25, 2.25, 2.26, 2.29, 2.31, 2.28, 2.32, 2.31, 2.21, 2.25, 2.25, 2.22, 2.24, 2.26, 2.15, 2.12, 2.1, 2.19, 2.3, 2.35, 2.34, 2.36, 2.36, 2.08, 2.06, 1.96, 1.96, 1.97, 2.12, 2.11, 2.11, 2.13, 2.12, 2.08, 2.23, 2.24, 2.17, 2.23, 2.26, 2.17, 2.32, 2.34, 2.31, 2.3, 2.28, 2.37, 2.65, 2.63, 2.51, 2.51, 2.48, 2.46, 2.46, 2.46]


/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_64237/1451715539.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Lets also look at the average volatility.

In [80]:
avg_volatility = np.round(np.mean(btc_volatility), 2)
print(f'Average volatility over 60 days: {avg_volatility}')

Average volatility over 60 days: 2.25


## Step 3: Plotting Volatility


Next, we will use Plotly to plot bitcoin's volatility in the past 60 days. The x-axis represents dates and the y-axis represents volatility (in percentage). The graph takes advantage of Plotly's hover functionality to allow users to see specific volatility values for specific dates.

The high level process for plotting volatitliy is the same as for plotting the coin closing prices. We still have to isolate the volatility scores and their dates and create a plotly figure object with a trace using the two lists. However isolating the volatitlty scores works a bit different.

As explained in step 2, in order to calculate daily volatility, we need to use a 30-day window of past prices. That means we have only volatitly scores for the 60 most recent days. So instead of coverting all 90 dates into a list we only use that last, or most recent, 60 dates using the .tail() function. .tail() is a pandas function that returns the last couple of values based on what number is inputted.

In [83]:
import plotly.graph_objects as go

def plot_volatility(coin, number_of_days=60):

    btc_opening_prices = daily_data[coin]['Open']

    dates = btc_opening_prices.tail(number_of_days).index.to_list()

    btc_volatility = volatility(btc_opening_prices, number_of_days, 30)

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=dates,
            y=btc_volatility,
            mode="lines+markers",      # line + scatter points
            name=f"{coin.capitalize()} Volatility"
        )
    )

    fig.update_layout(
        title=f"{coin.capitalize()} Volatility",
        xaxis_title="Date",
        yaxis_title="Volatility",
        hovermode="x unified"
    )

    fig.show()


plot_volatility("bitcoin", number_of_days=60)


/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_64237/1451715539.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



## Step 4 - Plotting Volatility for multiple coins

Using the new date and volatility isolation step for step 3, we can follow the exact same steps for plotting the closing prices for multiple coins.


In [84]:
import plotly.graph_objects as go

coins = ["bitcoin", "ethereum", "dogecoin"]


number_of_days = 60

fig = go.Figure()

for coin in coins:
    btc_opening_prices = daily_data[coin]['Open']
    dates = btc_opening_prices.tail(number_of_days).index.to_list()
    btc_volatility = volatility(btc_opening_prices, number_of_days, 30)

    fig.add_trace(
        go.Scatter(
            x=dates,
            y=btc_volatility,
            mode="lines+markers",
            name=f"{coin.capitalize()}"
        )
    )

fig.update_layout(
    title="Daily Volatility for Multiple Cryptocurrencies",
    xaxis_title="Date",
    yaxis_title="Volatility",
    hovermode="x unified"
)

fig.show()

/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_64237/1451715539.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



When investing in cryptocurrency, it is important to have a diverse portfolio to minimize risk of losing everything in one investment. Here, Plotly's hover functionality is particularly useful as it gives us information we can use to decide how much to allocate to each cryptocurrency. Advanced traders can profit off of high volatility but for beginners, it is best to allocate most of one's portfolio to cryptocurrencies with lower volatility.



# Section 5: Buy or Sell Quiz

Mock trades are a good way to practice making buy/sell decisions using real data and learn how to swing trade, which are trades that profit off of short-medium term investments that last a few days to a few weeks. That's why, we've built a game that challenges the player to make a buy/sell decision on a cryptocurrency, given a 15-day price chart, the risk/return ratio, and a volatility chart. The player will choose to either buy or sell the cryptocurrency at the closing price of the 15th day. If the player bought, they will sell 15 days later and see if they made a profit or lost money. If they sold, they will find out if they missed out on profit or made the right decision 15 days later.

In [92]:
import random

coins = ["bitcoin", "ethereum", "dogecoin"]

def game(coin_data, coins, investment_amount=100):

   coin = random.choice(coins)

   # Generate the initial 15-day price chart that the player gets to see before they make a buy/sell decision
   btc_data = coin_data[coin]
   prices = btc_data["Close"].to_list()
   dates = btc_data.index.to_list()
   mid = len(prices) // 2
   plot_close_prices(prices[:mid+1], dates[:mid+1], coin)
   initial_price = prices[:mid+1][-1]

   # provide player with risk/return and volatility info about the coin
   riskreturn = risk_return(prices[mid:])
   print(f"The risk/return ratio of {coin.capitalize()} is {riskreturn}.")
   plot_volatility(coin, 15)

   # establishing win and lose conditions
   answer = ''
   profit = prices[len(prices)-1]-prices[mid]
   if profit > 0:
       answer = 'buy'
   elif profit < 0:
       answer = 'sell'

   # we use a while loop to catch invalid inputs
   while True:
       guess = input(f"You have ${investment_amount} of {coin.capitalize()}. You are given a price chart, risk/return/value, and volatility chart for {coin} data from the past 15 days. Make a decision: buy or sell? (enter 'buy' or 'sell') ")
       if guess.lower() in ["buy", "sell"]:
           break
       else:
           print("Invalid input. Please type 'buy' or 'sell'")

   # price chart for 15 days after player bought or sold
   plot_close_prices(prices[mid:], dates[mid:], coin)
   final_price = prices[mid:][-1]

   # calculating price change in player's investment
   investment = round(investment_amount*(final_price/initial_price), 2)
   price_change = round(abs(investment - investment_amount), 2)

   # the player wins if after 15 days their decision leads to a profit
   if answer == '':
       print(f"The {coin} price has not changed in the past 15 days. You neither won nor lost money.")
   if answer == "buy":
       if guess.lower() == answer:
           print(f"Congrats! You made ${price_change}! Your new balance is ${investment}.")
       else:
           print(f"Unfortunately, you missed out on ${price_change} of profit. Better luck next time.")       
   elif answer == "sell":
       if guess.lower() == answer:
           print(f"Congrats! You saved ${price_change} by selling!")
       else:
           print(f"Unfortunately you lost ${price_change}. Your new balance is ${investment}.")

game(daily_data, coins)

The risk/return ratio of Ethereum is -1.05.


/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_64237/1451715539.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Congrats! You saved $20.37 by selling!


Throughout this tutorial, we built a complete workflow for analyzing cryptocurrencies using real market data. We started by fetching raw API responses and turning them into structured daily datasets, which let us equally compare Bitcoin, Ethereum, and Dogecoin. With this, we visualized price trends, calculated risk/return and volatility, and used those insights in the interactive buy/sell game.

More broadly, this tutorial shows users how to turn messy real-world data into meaningful financial analysis. The same process of fetch → clean → visualize → analyze can really be applied to any asset or dataset, giving learners a practical toolkit they can use far beyond the examples presented here.